In [88]:
!pip install tensorflow keras scikit-learn pandas

In [89]:
from google.colab import files
uploaded = files.upload()

Saving eda_123.csv to eda_123 (1).csv


In [118]:
import pandas as pd

data = pd.read_csv('eda_123.csv')
print(data.head())

   label                                               text
0      1  leftover the movie was visually stunning and l...
1      1  the movie was visually stupefy and left a last...
2      1  the movie was leftover visually stunning and l...
3      1  the movie was visually stunning and left a las...
4      1  last the movie was visually stunning and left ...


In [119]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

X = data['text']
y = data['label']

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_seq, maxlen=100, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=100, padding='post', truncating='post')

In [131]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras import regularizers

model = Sequential([

    Embedding(input_dim=696, output_dim=128, input_length=15),
    Conv1D(128, 5, activation='relu', kernel_regularizer=regularizers.l2()),
    Conv1D(16, 5, activation='relu', kernel_regularizer=regularizers.l2()),
    Conv1D(16, 5, activation='relu', kernel_regularizer=regularizers.l2()),
    Conv1D(8, 5, activation='relu', kernel_regularizer=regularizers.l2()),

    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.4),
    Dense(1, activation='sigmoid')
])
model.build(input_shape=(None, 200))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', 'precision', 'recall', 'auc'])

model.summary()

Model: "sequential_43"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_43 (Embedding)             │ (None, 200, 128)            │          89,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_152 (Conv1D)                  │ (None, 196, 128)            │          82,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_153 (Conv1D)                  │ (None, 192, 16)             │          10,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_154 (Conv1D)                  │ (None, 188, 16)             │           1,296 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_155 (Conv1D)                  │ (None, 184, 8)              │             648 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_43              │ (None, 8)                   │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_86 (Dense)                     │ (None, 64)                  │             576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_43 (Dropout)                 │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_87 (Dense)                     │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 183,977 (718.66 KB)

 Trainable params: 183,977 (718.66 KB)

 Non-trainable params: 0 (0.00 B)

In [132]:
history = model.fit(
    X_train_padded,
    y_train,
    epochs=8,
    batch_size=32,
    validation_data=(X_test_padded, y_test)
)

Epoch 1/8
26/26 ━━━━━━━━━━━━━━━━━━━━ 6s 137ms/step - accuracy: 0.5210 - auc: 0.5048 - loss: 2.1477 - precision: 0.5231 - recall: 0.9189 - val_accuracy: 0.5096 - val_auc: 0.5566 - val_loss: 1.3172 - val_precision: 0.5096 - val_recall: 1.0000
Epoch 2/8
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s 60ms/step - accuracy: 0.5998 - auc: 0.6475 - loss: 1.1799 - precision: 0.5687 - recall: 0.8373 - val_accuracy: 0.8798 - val_auc: 0.5000 - val_loss: 0.9051 - val_precision: 0.8785 - val_recall: 0.8868
Epoch 3/8
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - accuracy: 0.6283 - auc: 0.6842 - loss: 0.8649 - precision: 0.5903 - recall: 0.7789 - val_accuracy: 0.5577 - val_auc: 0.9123 - val_loss: 0.7834 - val_precision: 0.5354 - val_recall: 1.0000
Epoch 4/8
26/26 ━━━━━━━━━━━━━━━━━━━━ 2s 59ms/step - accuracy: 0.6889 - auc: 0.9088 - loss: 0.7656 - precision: 0.6231 - recall: 0.9881 - val_accuracy: 0.8413 - val_auc: 0.9872 - val_loss: 0.6762 - val_precision: 0.7626 - val_recall: 1.0000
Epoch 5/8
26/26 ━━━━━━━━━━━━━━━━━━━━ 3s

In [133]:
loss, accuracy ,precision, recall, auc = model.evaluate(X_test_padded, y_test)
print(f'Test Accuracy: {accuracy:.4f}')

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9748 - auc: 0.9967 - loss: 0.1680 - precision: 0.9856 - recall: 0.9679
Test Accuracy: 0.9808
